In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import skfda
import os
import sys
import random
import time
import tifffile
import shutil
import zarr
import csv
import numpy as np
import pandas as pd
import dask.array as da
from matplotlib import pyplot as plt
from typing import Union, List
from pathlib import Path
from tifffile import imread
from glob import glob
from tqdm import tqdm
from scipy import interpolate

from skimage import exposure
from skfda.representation.basis import FDataBasis
from skimage.io import imread, imshow, imsave
from skimage.transform import rescale, resize

import numpy as np
from scipy.signal import correlate
from scipy.interpolate import interp1d
import pickle

from exploratory_data_analysis import process_samples
from normalize import correlation_based_normalization, calculate_correlations_and_divergences, adjust_shifted_histograms, normalize_and_plot_distributions, plot_distributions
from image_transformation import calculate_shift_in_log_pixels, process_and_save_markers

import warnings
warnings.filterwarnings('ignore')

# Load tiff images as dask arrays for most efficient memory use

In [ ]:
# for loading image files (ome-tiff) to dask array
def tifffile_to_dask(im_fp: Union[str, Path]) -> Union[da.array, List[da.Array]]:
    imdata = zarr.open(tifffile.imread(im_fp, aszarr=True))
    if isinstance(imdata, zarr.hierarchy.Group):
        imdata = [da.from_zarr(imdata[z]) for z in imdata.array_keys()]
    else:
        imdata = da.from_zarr(imdata)
    return imdata

# Load your dataset

* you should change the file path accordingly over here

In [ ]:
## Directories
crc_dir = "/home/groups/ChangLab/dataset/Orion_CRC/"

# CRC 01 saved to different location
crc01_dir = '/home/groups/ChangLab/dataset/Orion_CRC/CRC01/registration'
crc01_fname = 'P37_S29_A24_C59kX_E15__at__20220106_014304_946511.ome.tiff'

# file names for all CRC slides in numeric order
crc_if_fnames = [
    os.path.join(crc01_dir, crc01_fname),
    os.path.join(crc_dir, 'CRC02/P37_S30_A24_C59kX_E15_20220106_014319_409148-zlib.ome.tiff'),
    os.path.join(crc_dir, 'CRC03/P37_S31_A24_C59kX_E15_20220106_014409_014236-zlib.ome.tiff'),
    os.path.join(crc_dir, 'CRC04/P37_S32_A24_C59kX_E15_20220106_014630_553652-zlib.ome.tiff'),
    os.path.join(crc_dir, 'CRC05/P37_S33_A24_C59kX_E15_20220107_180446_881530-zlib.ome.tiff'),
    os.path.join(crc_dir, 'CRC06/P37_S34_A24_C59kX_E15_20220107_202112_212579-zlib.ome.tiff')
                ]

# if your tiff images have several pyramid levels, then you should use the second resolution (which is 10X, 0.65um)
crc_dask_arrays = [tifffile_to_dask(fname)[1] for fname in crc_if_fnames]

# Read the metadata 

In [ ]:
with open(os.path.join(crc_dir+"CRC02", 'markers.csv'), newline = '') as f:
    reader = csv.reader(f)
    marker_list = [x[0] for x in list(reader)]

In [ ]:
sample_names = ["CRC01", "CRC02", "CRC03", "CRC04", "CRC05", "CRC06"]


tissue_mask_paths = {
    "CRC01": "/home/groups/ChangLab/dataset/Orion_CRC/CRC01/CRC01_reinhard_mask.tiff",
    "CRC02": "/home/groups/ChangLab/dataset/Orion_CRC/CRC02/CRC02_reinhard_mask.tiff",
    "CRC03": "/home/groups/ChangLab/dataset/Orion_CRC/CRC03/CRC03_reinhard_mask.tiff",
    "CRC04": "/home/groups/ChangLab/dataset/Orion_CRC/CRC04/CRC04_reinhard_mask.tiff",
    "CRC05": "/home/groups/ChangLab/dataset/Orion_CRC/CRC05/CRC05_reinhard_mask.tiff",
    "CRC06": "/home/groups/ChangLab/dataset/Orion_CRC/CRC06/CRC06_reinhard_mask.tiff",
}


xlims = [
    None,  # Limits for the first marker
    (4.5, 6.5),  # Limits for the second marker
    (4.8, 5.8),
    (4.75, 6.0),
    
    (5.0, 6.0),
    (5, 5.6),
    (4.75, 6),
    (5, 5.6),
    
    (5, 5.7),
    (4.5, 7),
    (5, 5.8),
    (4, 7),
    
    (4.75, 6.25), 
    (4.5, 6.5), 
    (4.5, 6.0), 
    (5, 6.0), 
    
    (5, 6),
    (4.5, 6.5),
    (4.5, 6.5) 
] 

ylims = [
    (0, 4e6),   # Limits for the first marker
] + [None] * 18  # Use full range for the remaining markers



# EDA to see varations in histograms

In [ ]:
# This is the command for when you do not use the tissue mask (using all pixels)
results_range, results_hist = process_samples(
    crc_dask_arrays=crc_dask_arrays, 
    sample_names=sample_names, 
    marker_list=marker_list, 
    bin_counts=1024, 
    subplots_per_row=4, 
    if_grid=True, 
    tissue_mask=True, 
    tissue_mask_paths=tissue_mask_paths,
    xlims=xlims,  # Define or leave as None if not using
    ylims=ylims,  # Define or leave as None if not using
    save_filename="crc01-04-all-markers-histograms-log-2nd-resolution.png"
)

file_path = 'crc01-06-all-marker-2nd-resolution-range-list.pkl'  # Change this to your desired file path
with open(file_path, 'wb') as file:
    pickle.dump(results_range, file)
    
file_path = 'crc01-06-all-marker-2nd-resolution-hist-list.pkl'  # Change this to your desired file path
with open(file_path, 'wb') as file:
    pickle.dump(results_hist, file)

# Perform normalization

In [ ]:
reference_sample = "CRC01"
bin_counts = 1024
xlim= [(300, 700) for _ in range(19)]

# Normalize and plot distributions for each key
shifts_fft_dict = normalize_and_plot_distributions(results_hist, marker_list, sample_names, reference_sample, bin_counts, xlim)

# Perform image transformation

In [ ]:
# Calculate shift in log pixels
shift_in_log_pixels_dict = calculate_shift_in_log_pixels(results_range, marker_list, bin_counts, shifts_fft_dict)

output_directory = "/home/groups/ChangLab/wangmar/shift-panel-reduction/shift-panel-reduction-main/landmark_normalization/cycif-normalization-pipeline"
reference_sample = "CRC01"
reference_index = sample_names.index(reference_sample)

# save stacked images
process_and_stack_images(crc_dask_arrays, marker_list, shift_in_log_pixels_dict, reference_index, bin_counts, plot_dist=True, output_directory)